[View in Colaboratory](https://colab.research.google.com/github/AlbertZheng/quickdraw-cnn/blob/master/quickdraw_cnn.ipynb)

## "You draw, I guess." is a MVP (Minimium Viable Product) that uses CNN to recognize the sketch drawings on web canvas.
### The CNN was trained to recognize 10 classes using <a href='https://github.com/googlecreativelab/quickdraw-dataset'>"The Quick, Draw! Dataset" </a> of Google awesome "猜画小歌" Wechat App.

## Install dependent packages

In [1]:
!pip install "tensorlayer>=1.10"
!pip install tensorflowjs
!pip list|grep tensor

    100% |████████████████████████████████| 317kB 6.1MB/s 
    100% |████████████████████████████████| 31.2MB 1.2MB/s 
    100% |████████████████████████████████| 25.3MB 1.7MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 12.6MB 4.4MB/s 
    100% |████████████████████████████████| 5.8MB 4.2MB/s 
    100% |████████████████████████████████| 3.3MB 6.1MB/s 
    100% |████████████████████████████████| 92kB 25.6MB/s 
    100% |████████████████████████████████| 2.0MB 13.0MB/s 
    100% |████████████████████████████████| 675kB 14.3MB/s 
    100% |████████████████████████████████| 952kB 18.9MB/s 
  Running setup.py bdist_wheel for imageio ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/e0/43/31/605de9372ceaf657f152d3d5e82f42cf265d81db8bbe63cde1
Successfully built imageio
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
  Found existing installatio

## Import dependences, and check if GPU is available

In [5]:
import os
import time
import urllib.request
import urllib.parse
import numpy as np
from random import randint

import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
from tensorflow.python import debug as tfdebug

""" Notice to put ```import matplotlib.pyplot``` after imports of tensorlayer, 
otherwise you will get below warning:

This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.
"""
import matplotlib.pyplot as plt

device_name = tf.test.gpu_device_name()
print('### device name: {} ###'.format(device_name))
if device_name != '/device:GPU:0':
    raise SystemError('*** GPU device not found ***')
print('### Found GPU at: {} ###'.format(device_name))


tf.logging.set_verbosity(tf.logging.DEBUG)
tl.logging.set_verbosity(tl.logging.DEBUG)

config = tf.ConfigProto()
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config.gpu_options.allow_growth = True
config.allow_soft_placement = True


### device name: /device:GPU:0 ###
### Found GPU at: /device:GPU:0 ###


## Download the Quick, Draw dataset

In [6]:
working_directory = 'data'
dataset_directory = 'data/quickdraw'
# categories_filename = 'categories.txt'
# categories_file_url_source = 'https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/'

# npy_dataset_url_source = 'https://storage.cloud.google.com/quickdraw_dataset/full/numpy_bitmap/'
npy_dataset_url_source = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'

X = "X"
y = "y"

X_NUMPY_DTYPE = np.float32
y_NUMPY_DTYPE = np.int64
X_TF_DTYPE = tf.float32
y_TF_DTYPE = tf.int32

image_height = 28
image_width = 28
image_depth = 1
image_size = image_height * image_width * image_depth
input_layer_X_shape = [image_height, image_width, image_depth]
input_layer_X_shape_batch = [-1, image_height, image_width, image_depth]

mini_categories_file_url_source = 'https://raw.githubusercontent.com/AlbertZheng/quickdraw-cnn/master/web/'
mini_categories_filename = 'mini-categories.txt'
n_category = 10  # The maximum of category number is up to 345
n_train_example_per_category = 100000


def print_dataset_shape(X_name, X, y_name, y):
    print(X_name + '.shape ', X.shape, end='\t,\t')
    print(y_name + '.shape ', y.shape)
    print('%s.dtype %s\t,\t%s.dtype %s' % (X_name, X.dtype, y_name, y.dtype))


def show_image(X, y, categories):
    plt.imshow(X.reshape(image_height, image_width), cmap="gray", interpolation='nearest')
    plt.title(f"{categories[y]}(label: {y})")
    plt.show()


def load_quickdraw_dataset(
    n_category=10, n_train_example_per_category=20000
):
    """ Download the quick draw data set. """
    n_validation_example_per_category = int(n_train_example_per_category / 0.7 * 0.2)
    n_test_example_per_category = int(n_train_example_per_category / 0.7 * 0.1)

    # Download the categories file
    tl.files.utils.maybe_download_and_extract(mini_categories_filename, dataset_directory, mini_categories_file_url_source)

    tl.logging.info("Load or Download quick draw > {}".format(dataset_directory))

    train_set = {X: np.empty([0, image_size], dtype=X_NUMPY_DTYPE), y: np.empty([0], dtype=y_NUMPY_DTYPE)}
    validation_set = {X: np.empty([0, image_size], dtype=X_NUMPY_DTYPE), y: np.empty([0], dtype=y_NUMPY_DTYPE)}
    test_set = {X: np.empty([0, image_size], dtype=X_NUMPY_DTYPE), y: np.empty([0], dtype=y_NUMPY_DTYPE)}

    category_names = [line.rstrip('\n') for line in open(f"{dataset_directory}/{mini_categories_filename}")]
    for category_index, category_name in enumerate(category_names):
        if category_index == n_category:
            break

        category_names[category_index], _, _ = category_name.rpartition('=')
        category_name = category_names[category_index]

        filename = urllib.parse.quote(category_name) + '.npy'
        tl.files.utils.maybe_download_and_extract(filename, dataset_directory, npy_dataset_url_source)

        data = np.load(os.path.join(dataset_directory, filename))
        size_per_category = data.shape[0]
        labels = np.full(size_per_category, category_index)

        print(f"### Category '{category_name}' id:{category_index} dataset info ###")
        print_dataset_shape("data", data, "labels", labels)

        number_begin = 0
        number_end = n_train_example_per_category
        # train_set[X] = np.concatenate((train_set[X], data[number_begin: number_end, :]), axis=0)
        train_set[X] = np.vstack((train_set[X], data[number_begin: number_end, :]))
        train_set[y] = np.append(train_set[y], labels[number_begin: number_end])

        number_begin += n_train_example_per_category
        number_end += n_validation_example_per_category
        # validation_set[X] = np.concatenate((validation_set[X], data[number_begin:number_end, :]), axis=0)
        validation_set[X] = np.vstack((validation_set[X], data[number_begin:number_end, :]))
        validation_set[y] = np.append(validation_set[y], labels[number_begin:number_end])

        number_begin += n_validation_example_per_category
        number_end += n_test_example_per_category
        # test_set[X] = np.concatenate((test_set[X], data[number_begin:number_end, :]), axis=0)
        test_set[X] = np.vstack((test_set[X], data[number_begin:number_end, :]))
        test_set[y] = np.append(test_set[y], labels[number_begin:number_end])

        print_dataset_shape("train_set[X]", train_set[X], "train_set[y]", train_set[y])
        print_dataset_shape("validation_set[X]", validation_set[X], "validation_set[y]", validation_set[y])
        print_dataset_shape("test_set[X]", test_set[X], "test_set[y]", test_set[y])

    # Randomize the dataset
    size_per_set = train_set[X].shape[0]
    permutation = np.random.permutation(size_per_set)
    train_set[X] = train_set[X][permutation, :]
    train_set[y] = train_set[y][permutation]

    size_per_set = validation_set[X].shape[0]
    permutation = np.random.permutation(size_per_set)
    validation_set[X] = validation_set[X][permutation, :]
    validation_set[y] = validation_set[y][permutation]

    size_per_set = test_set[X].shape[0]
    permutation = np.random.permutation(size_per_set)
    test_set[X] = test_set[X][permutation, :]
    test_set[y] = test_set[y][permutation]

    # Reshape for CNN input
    train_set[X] = train_set[X].reshape(input_layer_X_shape_batch)
    validation_set[X] = validation_set[X].reshape(input_layer_X_shape_batch)
    test_set[X] = test_set[X].reshape(input_layer_X_shape_batch)

    # The original grayscale image is 'black background (x==0) and gray~white (0< x <=255) brush'
    # Because the CNN model doesn't need to learn the grayscale values and it only needs to
    # learn the strokes, we normalize it to 'white background (x==1) and block (x==0) brush'.
    train_set[X] = 1.0 - np.ceil(train_set[X] / 255.0)
    validation_set[X] = 1.0 - np.ceil(validation_set[X] / 255.0)
    test_set[X] = 1.0 - np.ceil(test_set[X] / 255.0)

    return category_names, train_set, validation_set, test_set


# Open TensorBoard logs writer
tfboard_file_writer = tf.summary.FileWriter('logs')

# Download data
category_names, train_set, validation_set, test_set = load_quickdraw_dataset(n_category, n_train_example_per_category)


[TL] Load or Download quick draw > data/quickdraw
### Category 'airplane' id:0 dataset info ###
data.shape  (151623, 784)	,	labels.shape  (151623,)
data.dtype uint8	,	labels.dtype int64
train_set[X].shape  (100000, 784)	,	train_set[y].shape  (100000,)
train_set[X].dtype float32	,	train_set[y].dtype int64
validation_set[X].shape  (28571, 784)	,	validation_set[y].shape  (28571,)
validation_set[X].dtype float32	,	validation_set[y].dtype int64
test_set[X].shape  (14285, 784)	,	test_set[y].shape  (14285,)
test_set[X].dtype float32	,	test_set[y].dtype int64
### Category 'alarm clock' id:1 dataset info ###
data.shape  (123399, 784)	,	labels.shape  (123399,)
data.dtype uint8	,	labels.dtype int64
train_set[X].shape  (200000, 784)	,	train_set[y].shape  (200000,)
train_set[X].dtype float32	,	train_set[y].dtype int64
validation_set[X].shape  (51970, 784)	,	validation_set[y].shape  (51970,)
validation_set[X].dtype float32	,	validation_set[y].dtype int64
test_set[X].shape  (14285, 784)	,	test_set[y]

## Function: Network model definition

In [0]:
def model_batch_normalization(X_batch, y_batch, output_units, reuse, is_train):
    """ Define the network model """
    W_init1 = tf.truncated_normal_initializer(stddev=5e-2)
    W_init2 = tf.truncated_normal_initializer(stddev=0.04)
    bias_init = tf.constant_initializer(value=0.1)

    with tf.variable_scope("model", reuse=reuse):
        net = InputLayer(X_batch, name='input')
        net = Conv2d(net, 128, (5, 5), (1, 1), padding='SAME',
                     W_init=W_init1, b_init=None, name='cnn1')
        net = BatchNormLayer(net, is_train, act=tf.nn.relu, name='batch1')
        net = MaxPool2d(net, (3, 3), (2, 2), padding='SAME', name='pool1')

        net = Conv2d(net, 256, (5, 5), (1, 1), padding='SAME',
                     W_init=W_init1, b_init=None, name='cnn2')
        net = BatchNormLayer(net, is_train, act=tf.nn.relu, name='batch2')
        net = MaxPool2d(net, (3, 3), (2, 2), padding='SAME', name='pool2')

        net = FlattenLayer(net, name='flatten')
        net = DenseLayer(net, 384, act=tf.nn.relu,
                         W_init=W_init2, b_init=bias_init, name='d1relu')
        net = DenseLayer(net, 192, act=tf.nn.relu,
                         W_init=W_init2, b_init=bias_init, name='d2relu')
        # The softmax() is implemented internally in tl.cost.cross_entropy(y, y_) to
        # speed up computation, so we use identity here.
        # see tf.nn.sparse_softmax_cross_entropy_with_logits()
        net = DenseLayer(net, n_units=output_units, act=None,
                         W_init=W_init2, name='output')

        y_prediction_batch_without_softmax = net.outputs

        # For inference by using this model
        # y_output = tf.argmax(tf.nn.softmax(y_prediction_batch_without_softmax), 1)
        y_output = tf.nn.softmax(y_prediction_batch_without_softmax, name="y_output")

        ce = tl.cost.cross_entropy(y_prediction_batch_without_softmax, y_batch, name='cost')

        """ 需给后面的全连接层引入L2 normalization，惩罚模型的复杂度，避免overfitting """
        # L2 for the MLP, without this, the accuracy will be reduced by 15%.
        L2 = 0
        for p in tl.layers.get_variables_with_name('relu/W', True, True):
            L2 += tf.contrib.layers.l2_regularizer(0.004)(p)
        # 加上L2模型复杂度惩罚项后，得到最终真正的cost
        cost = ce + L2

        correct_prediction = tf.equal(tf.cast(tf.argmax(y_prediction_batch_without_softmax, 1), y_TF_DTYPE), y_batch)
        # correct_prediction = tf.Print(correct_prediction, [correct_prediction], "correct_prediction: ")
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        return net, cost, accuracy, y_output


##  Train, validate and test

In [8]:
# Train, validate and test
batch_size = 128
n_epoch = 25
n_step_per_epoch = int(len(train_set[y]) / batch_size)
n_step = n_epoch * n_step_per_epoch
print_freq = 1
checkpoint_freq = 3
learning_rate = 0.0001

model_ckpt_file_name = os.path.join(working_directory, "checkpoint", "model-quickdraw-cnn.ckpt")
resume = True  # load model, resume from previous checkpoint?


def distort_fn(X, is_train=False):
    # print('begin', X.shape, np.min(X), np.max(X))

    if is_train == True:
        # 1. Randomly flip the image horizontally.
        X = tf.image.random_flip_left_right(X)

    # X = tf.image.per_image_standardization(X)

    # print('after norm', X.shape, np.min(X), np.max(X), np.mean(X))
    return X


def save_model():
    model_type = "saved-model"
    latest_model_directory = f'{model_type}-{time.strftime("%Y%m%d%H%M%S", time.localtime())}'
    saved_model_directory = os.path.join(working_directory, latest_model_directory)
    if not os.path.exists(saved_model_directory):
        tf.saved_model.simple_save(session, saved_model_directory,
                                   inputs={"X": X_batch_ph},
                                   outputs={"y_output": y_prediction_})
        dist_directory = os.path.join(".", model_type)
        if os.path.exists(dist_directory):
            os.remove(dist_directory)
        os.symlink(saved_model_directory, dist_directory, target_is_directory=True)


with tf.device('/cpu:0'):
    session = tf.Session(config=config)

    #
    # Connect to tfdbg dashboard by ```http://localhost:6006#debugger```
    # when the following command is issued.
    #
    # ```bash
    # $ tensorboard --logdir logs --port 6006 --debugger_port 6064
    # ```
    #
    # session = tfdebug.TensorBoardDebugWrapperSession(session, "albert-mbp.local:6064")

    X_batch_ph = tf.placeholder(dtype=X_TF_DTYPE, shape=[None, image_height, image_width, image_depth], name='X_batch')
    y_batch_ph = tf.placeholder(dtype=y_TF_DTYPE, shape=[None], name='y_batch')
    # X_batch_ph = tf.placeholder(dtype=X_TF_DTYPE, shape=[batch_size, image_height, image_width, image_depth], name='X')
    # y_batch_ph = tf.placeholder(dtype=y_TF_DTYPE, shape=[batch_size], name='y')

    def perform_minibatch(run_list, X, y, batch_size, is_train=False):
        n_batch, sum_loss, sum_accuracy = 0, 0, 0
        for X_batch_a, y_batch_a in tl.iterate.minibatches(X, y, batch_size, shuffle=is_train):
            # data augmentation for training
            # X_batch_a = tl.prepro.threading_data(X_batch_a, fn=distort_fn, is_train=is_train)

            cost, accuracy = 0, 0
            if is_train:
                _, cost, accuracy = session.run(
                    run_list, feed_dict={X_batch_ph: X_batch_a, y_batch_ph: y_batch_a}
                )
            else:
                cost, accuracy = session.run(
                    run_list, feed_dict={X_batch_ph: X_batch_a, y_batch_ph: y_batch_a}
                )

            sum_loss += cost
            sum_accuracy += accuracy
            n_batch += 1
        return n_batch, sum_loss, sum_accuracy


    with tf.device('/gpu:0'):  # <-- remove it if you don't have GPU
        # Build the model
        print("### Train Network model ###")
        network_, cost_, accuracy_, y_prediction_ = model_batch_normalization(
            X_batch_ph, y_batch_ph, n_category, reuse=None, is_train=True
        )
        print("### Reuse this Train Network model for validation and test ###")
        _, cost_test_, accuracy_test_, y_prediction_test_ = model_batch_normalization(
            X_batch_ph, y_batch_ph, n_category, reuse=True, is_train=False
        )

    # Define the training optimizer
    with tf.device('/gpu:0'):  # <-- remove it if you don't have GPU
        train_op_ = tf.train.AdamOptimizer(learning_rate).minimize(cost_)

    tl.layers.initialize_global_variables(session)

    # Attach the graph for TensorBoard writer
    # tfboard_file_writer.add_graph(tf.get_default_graph())
    tfboard_file_writer.add_graph(session.graph)

    if resume and os.path.isfile(model_ckpt_file_name):
        print("Load existing model " + "!" * 10)
        saver = tf.train.Saver()
        saver.restore(session, model_ckpt_file_name)

    print("### Network parameters ###")
    network_.print_params(False)
    print("### Network layers ###")
    network_.print_layers()

    print('   learning_rate: %f' % learning_rate)
    print('   batch_size: %d' % batch_size)
    print('   n_epoch: %d, step in an epoch: %d, total n_step: %d' % (n_epoch, n_step_per_epoch, n_step))

    step, sum_batch, sum_loss, sum_accuracy = 0, 0, 0, 0
    for epoch in range(n_epoch):
        start_time = time.time()

        n_batch_a_epoch, cost_a_epoch, accuracy_a_epoch = perform_minibatch(
            [train_op_, cost_, accuracy_],
            train_set[X], train_set[y], batch_size, is_train=True
        )
        sum_batch += n_batch_a_epoch
        sum_loss += cost_a_epoch
        sum_accuracy += accuracy_a_epoch
        step += n_batch_a_epoch

        assert n_batch_a_epoch == n_step_per_epoch

        if epoch + 1 == 1 or (epoch + 1) % print_freq == 0:
            print("Epoch %d : Step %d-%d of %d took %fs" %
                  (epoch + 1, step - n_step_per_epoch, step, n_step, time.time() - start_time))
            print("   train loss: %f" % (sum_loss / sum_batch))
            print("   train accuracy: %f" % (sum_accuracy / sum_batch))
            sum_batch, sum_loss, sum_accuracy = 0, 0, 0

            n_batch_a_epoch, cost_a_epoch, accuracy_a_epoch = perform_minibatch(
                [cost_test_, accuracy_test_],
                validation_set[X], validation_set[y], batch_size
            )
            print("   validation loss: %f" % (cost_a_epoch / n_batch_a_epoch))
            print("   validation accuracy: %f" % (accuracy_a_epoch / n_batch_a_epoch))

            n_batch_a_epoch, cost_a_epoch, accuracy_a_epoch = perform_minibatch(
                [cost_test_, accuracy_test_],
                test_set[X], test_set[y], batch_size
            )
            print("   test loss: %f" % (cost_a_epoch / n_batch_a_epoch))
            print("   test accuracy: %f" % (accuracy_a_epoch / n_batch_a_epoch))

        # Save model when checkpoint
        if (epoch + 1) % checkpoint_freq == 0:
            print("Saving checkpoint... " + "!" * 10)
            saver = tf.train.Saver()
            save_path = saver.save(session, model_ckpt_file_name)
            print("Saving model... " + "!" * 10)
            save_model()


### Train Network model ###
[TL] InputLayer  model/input: (?, 28, 28, 1)
[TL] Conv2d model/cnn1: n_filter: 128 filter_size: (5, 5) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer model/batch1: decay: 1.000000 epsilon: 0.000010 act: relu is_train: False
[TL] MaxPool2d model/pool1: filter_size: (3, 3) strides: (2, 2) padding: SAME
[TL] Conv2d model/cnn2: n_filter: 256 filter_size: (5, 5) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer model/batch2: decay: 1.000000 epsilon: 0.000010 act: relu is_train: False
[TL] MaxPool2d model/pool2: filter_size: (3, 3) strides: (2, 2) padding: SAME
[TL] FlattenLayer model/flatten: 12544
[TL] DenseLayer  model/d1relu: 384 relu
[TL] DenseLayer  model/d2relu: 192 relu
[TL] DenseLayer  model/output: 10 No Activation
[TL]   [*] geting variables with relu/W
[TL]   got   0: model/d1relu/W:0   (12544, 384)
[TL]   got   1: model/d2relu/W:0   (384, 192)
### Reuse this Train Network model for validation and test ###
[TL] InputLay

## Save the trained model

In [9]:
save_model()

tfboard_file_writer.flush()
tfboard_file_writer.close()

session.close()


INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: data/saved-model-20181016074029/saved_model.pb


## Convert to TensorFlow.js web model

In [10]:
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names="model/y_output" saved-model web-model
!echo "Current directory ->"
!ls -la
!echo "web-model directory ->"
!ls -la web-model
!echo "saved-model directory ->"
!ls -la saved-model

Using TensorFlow backend.
2018-10-16 07:40:46.982535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-10-16 07:40:46.983029: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 8.92GiB
2018-10-16 07:40:46.983090: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-10-16 07:40:47.443933: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-10-16 07:40:47.444010: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2018-10-16 07:40:47.444065: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N 
2018-10-16 07:40:47.444423: W tensorflow/core/common_runtime/

## Zip and download the models

In [11]:
!zip -r web-model.zip web-model
!zip -r saved-model.zip saved-model
!ls -la *.zip

from google.colab import files
files.download('web-model.zip')
files.download('saved-model.zip')


  adding: web-model/ (stored 0%)
  adding: web-model/weights_manifest.json (deflated 72%)
  adding: web-model/group1-shard6of6 (deflated 8%)
  adding: web-model/group1-shard5of6 (deflated 7%)
  adding: web-model/group1-shard1of6 (deflated 7%)
  adding: web-model/group1-shard4of6 (deflated 7%)
  adding: web-model/tensorflowjs_model.pb (deflated 7%)
  adding: web-model/group1-shard2of6 (deflated 7%)
  adding: web-model/group1-shard3of6 (deflated 7%)
  adding: saved-model/ (stored 0%)
  adding: saved-model/variables/ (stored 0%)
  adding: saved-model/variables/variables.data-00000-of-00001 (deflated 30%)
  adding: saved-model/variables/variables.index (deflated 46%)
  adding: saved-model/saved_model.pb (deflated 91%)
-rw-r--r-- 1 root root 48361888 Oct 16 07:42 saved-model.zip
-rw-r--r-- 1 root root 24228564 Oct 16 07:41 web-model.zip


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 35264, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil